## Notebook Imports

In [170]:
import pandas as pd
import numpy as np

## Constant

In [171]:
TRAINING_DATA_FILE = 'SpamData1/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData1/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData1/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData1/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData1/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData1/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData1/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## Read and Load Features from .txt Files into Numpy Array

In [172]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [173]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [174]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in test file', sparse_test_data.shape[0])

Nr of rows in training file 265427
Nr of rows in test file 110522


In [175]:
print('Nr of emails in training file', np.unique(sparse_train_data[:]).size)

Nr of emails in training file 4832


## How to create a DataFrame

In [176]:
column_names= ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [177]:
len(column_names)

2502

In [178]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [179]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
full_train_data.fillna(value=0, inplace=True) # remember inplace serves as instead of using same var to overide

In [181]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [182]:
def make_full_matrix(sparse_matrix,nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    
    form a full matrix from a sparse matrix. return a pandas dataframe
    keyword arguments:
    sparse matrix -- numpy array
        sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix


In [183]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: total: 23.2 s
Wall time: 23.3 s


In [184]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


In [185]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,2,2,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5791,0,2,0,1,1,0,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,2,2,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5795,0,1,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Naive Bayes Model

### Calculating the Probability of Spam

In [186]:
# Challenge: Calculate the probability of spam - the percent of spam messages in the training
# dataset. Store this value in a variable called prob_spam

In [187]:
full_train_data.CATEGORY.size

4015

In [188]:
 full_train_data.CATEGORY.sum() 

1250

In [189]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size

In [190]:
print('Probability of spam is', prob_spam)

Probability of spam is 0.31133250311332505


## Total Number of Words / Tokens

In [191]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [192]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4015,)

In [193]:
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    136
dtype: int64

In [194]:
total_wc= email_lengths.sum()
total_wc

445891

## Number of Tokens in spam & ham Emails

**Challenge** Create a subset of the email_lengths series that only contains the spam messages. Call the subset spam_lengths. Then count the total number of words that occur in spam emails. 

Do the same for the non-spam emails. Create a subset called ham_lengths. Then count the total number of words that occur in the ham emails. 

In [195]:
spam_lengths= email_lengths[full_train_data.CATEGORY==1]
spam_lengths.shape

(1250,)

In [196]:
spam_wc=spam_lengths.sum()
spam_wc

195645

In [197]:
ham_lengths= email_lengths[full_train_data.CATEGORY==0]
ham_lengths.shape

(2765,)

In [198]:
email_lengths.shape[0]- spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [199]:
nonspam_wc= ham_lengths.sum()
nonspam_wc

250246

In [200]:
spam_wc + nonspam_wc-total_wc

0

In [201]:
# Challenge: Can you figure out if spam emails or non-spam emails tend to be longer? Which
# category contains more tokens? Take a guess and then verify your intuition using the dataset

In [202]:
print('Average nr of words in spam emails {:.0f} '.format(spam_wc/spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f} '.format(nonspam_wc/ham_lengths.shape[0]))

Average nr of words in spam emails 157 
Average nr of words in ham emails 90.505 


## Summing the Tokens Occuring in Spam

In [203]:
full_train_features.shape

(4015, 2500)

In [204]:
train_spam_tokens= full_train_features.loc[full_train_data.CATEGORY ==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [205]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,2,0,0,3,2,2,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,2,0,1,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [207]:
summed_spam_tokens.tail()

2495    19
2496     2
2497    23
2498    10
2499     4
dtype: int64

### Summing the Tokens Occuring in Ham

In [208]:
train_ham_tokens= full_train_features.loc[full_train_data.CATEGORY ==0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,0,0,0,0,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1898,0,3,1,0,2,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1899,0,1,0,0,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1900,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1901,2,0,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [209]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape

(2500,)

In [210]:
summed_ham_tokens.tail()

2495    13
2496    19
2497     1
2498    16
2499    35
dtype: int64

In [211]:
train_ham_tokens[2499].sum() + 1

35

## P(Token | Spam) - Probability that a Token Occurs given the Email is spam

In [212]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.009049
1    0.004885
2    0.006838
3    0.010593
4    0.006803
dtype: float64

In [213]:
prob_tokens_spam.sum()

1.0

## P(Token | ham) - Probability that a Token Occurs given the Email is ham

In [214]:
prob_tokens_ham = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_ham[:5]

0    0.020863
1    0.009887
2    0.008063
3    0.003735
4    0.006311
dtype: float64

In [215]:
prob_tokens_ham.sum()

1.0

## P(Token) - Probability that Token Occurs

In [216]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [217]:
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [218]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

**Challenge:** Create a full matrix from the sparse_test_data. Time the function call. How long does it take? Separate the features and the target values. Save these as separate .txt files: a TEST_TARGET_FILE and a TEST_FEATURE_MATRIX file. 

In [219]:
sparse_test_data.shape

(110522, 4)

In [220]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: total: 12.6 s
Wall time: 13.4 s


In [221]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY' ]
y_test = full_test_data.CATEGORY

In [222]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)